<a href="https://colab.research.google.com/github/s2hee/2022_Summer_intern/blob/main/Assignment1_ResNet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Data Load

In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"se2heee","key":"50bb79e8f964d24bad4b83fb36d31f97"}'}

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d swaroopkml/cifar10-pngs-in-folders

 85% 119M/140M [00:00<00:00, 281MB/s] 
100% 140M/140M [00:00<00:00, 244MB/s]


In [5]:
!ls

cifar10-pngs-in-folders.zip  drive  kaggle.json  sample_data


In [6]:
!unzip /content/cifar10-pngs-in-folders.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: cifar10/cifar10/train/truck/0001.png  
  inflating: cifar10/cifar10/train/truck/0002.png  
  inflating: cifar10/cifar10/train/truck/0003.png  
  inflating: cifar10/cifar10/train/truck/0004.png  
  inflating: cifar10/cifar10/train/truck/0005.png  
  inflating: cifar10/cifar10/train/truck/0006.png  
  inflating: cifar10/cifar10/train/truck/0007.png  
  inflating: cifar10/cifar10/train/truck/0008.png  
  inflating: cifar10/cifar10/train/truck/0009.png  
  inflating: cifar10/cifar10/train/truck/0010.png  
  inflating: cifar10/cifar10/train/truck/0011.png  
  inflating: cifar10/cifar10/train/truck/0012.png  
  inflating: cifar10/cifar10/train/truck/0013.png  
  inflating: cifar10/cifar10/train/truck/0014.png  
  inflating: cifar10/cifar10/train/truck/0015.png  
  inflating: cifar10/cifar10/train/truck/0016.png  
  inflating: cifar10/cifar10/train/truck/0017.png  
  inflating: cifar10/cifar10/train/truck/0018.png  
  inflating: cifar10/cifar10

# 1. Preparing

In [7]:
import numpy as np 
import pandas as pd 
import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader 
import os
import matplotlib.pyplot as plt
import csv

In [8]:
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.optim import Adam
from torch.autograd import Variable

In [9]:
from torch.autograd import Variable
from torch.optim import lr_scheduler
import torchvision.transforms as tr
import torchvision.models as models
from torchvision.models import vgg16

In [16]:
batch_size = 32 
epochs = 10 
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

transf_train = tr.Compose([tr.RandomCrop(32, padding=4), tr.ToTensor(),tr.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transf_test = tr.Compose([tr.ToTensor(),tr.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])


# CIFAR10 custom dataset
trainset = torchvision.datasets.ImageFolder(root='/content/cifar10/cifar10/train',transform=transf_train)
testset = torchvision.datasets.ImageFolder(root='/content/cifar10/cifar10/test',transform=transf_test)

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

# 2. ResNet Model

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [12]:
def train(model, trainloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, 
                batch_idx * len(data), 
                len(trainloader.dataset), 
                100. * batch_idx / len(trainloader), 
                loss.item()))

In [13]:
def evaluate(model, testloader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction = "sum").item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()
    
    test_loss /= len(testloader.dataset)
    test_accuracy = 100. * correct / len(testloader.dataset)
    return test_loss, test_accuracy

In [15]:
import torchvision.models as models
model = models.resnet18().cuda()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
epochs = 20
for epoch in range(1, epochs + 1):
    train(model, trainloader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, testloader)
    print("[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 6.815615
Train Epoch: 1 [3200/50000 (6%)]	Loss: 1.919024
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.749847
Train Epoch: 1 [9600/50000 (19%)]	Loss: 1.643875
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.429716
Train Epoch: 1 [16000/50000 (32%)]	Loss: 1.478291
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.777019
Train Epoch: 1 [22400/50000 (45%)]	Loss: 1.523023
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.639494
Train Epoch: 1 [28800/50000 (58%)]	Loss: 1.515646
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.141811
Train Epoch: 1 [35200/50000 (70%)]	Loss: 1.564785
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.405704
Train Epoch: 1 [41600/50000 (83%)]	Loss: 1.340050
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.116232
Train Epoch: 1 [48000/50000 (96%)]	Loss: 1.123654
[1] Test Loss: 1.2463, accuracy: 55.31%

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.293985
Train Epoch: 2 [3200/50000 (6%)]	Loss: 1.445415
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.284801
Train Epoch: 2 [9600/50000